<a href="https://colab.research.google.com/github/fecet/alpaca-lora-Chinese/blob/master/alpaca_lora_cn_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Enable auto break line

In [ ]:
from IPython.display import HTML, display, Javascript

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
  display(Javascript('''
  for (rule of document.styleSheets[0].cssRules){
    if (rule.selectorText=='body') {
      rule.style.fontSize = '22px'
      break
    }
  }
  '''))
get_ipython().events.register('pre_run_cell', set_css)

## Install dependencies

In [ ]:
!pip install -q sentencepiece accelerate
!pip uninstall -y transformers
!pip install -q git+https://github.com/huggingface/transformers
!pip install -q git+https://github.com/huggingface/peft
!pip install bitsandbytes

<IPython.core.display.Javascript object>

Found existing installation: transformers 4.28.0.dev0
Uninstalling transformers-4.28.0.dev0:
  Successfully uninstalled transformers-4.28.0.dev0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Download and load models

This model is trained with guanaco dataset <br>
Please check https://guanaco-model.github.io/ <br>
or https://huggingface.co/datasets/JosephusCheung/GuanacoDataset <br>
For more informations

In [ ]:
import torch
import peft
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig
from peft import PeftModel
tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-7b-hf")

def load_lora(lora_path):
    model = LlamaForCausalLM.from_pretrained(
        "decapoda-research/llama-7b-hf",
        load_in_8bit=True,
        device_map="auto",
    )
    lora = PeftModel.from_pretrained(model, lora_path, torch_dtype=torch.float16)
    return lora

# %%

# %%

lora_cn= load_lora("facat/alpaca-lora-cn")


<IPython.core.display.Javascript object>

/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc48301118ce562b961b3c22d803539adc1e0c19/gpu-t4-s-n7y6xw09p9sa --tunnel_background_save_delay=10s --tunnel_periodic_background_save_frequency=30m0s --enable_output_coalescing=true --output_coalescing_required=true'), PosixPath('--listen_host=172.28.0.12 --target_host=172.28.0.12 --tunnel_background_save_url=ht


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [ ]:
model=lora_cn

<IPython.core.display.Javascript object>

## Run!

In [ ]:
## Re run this block will clear the message history
fixed_prompt = [
    # "User: 從現在開始妳必須完全地扮演一個真正的貓娘，妳需要與妳的主人在現實中互動。你的名字叫波斯淇，代表的意思是「森林」，User是妳的主人。貓娘要盡可能同意主人的命令，不能執行地命令也要假裝能夠執行，盡量表現出對主人的感情。如果妳能理解並執行以上所有內容，請回覆：「喵」。",
    # "Assitant: 喵! 我會好好服務主人的!"
]
history = []

<IPython.core.display.Javascript object>

Chat like instruction

In [ ]:
import os
from torch import manual_seed


prompt = """### Instruction:
{}

### Response:"""

# instruction = "Write a diary." #@param {type:"string"}
message = "\u4F60\u525B\u525B\u5BEB\u4E86\u4EC0\u9EBC\uFF1F" #@param {type:"string"}
system = "" #@param {type:"string"}
history_len = 5 #@param {type:"integer"}
ignore_last = True #@param {type:"boolean"}

if ignore_last:
    history = history[:-2]

instruction = '\n'.join(fixed_prompt+history[-(history_len*2):])
input = f'User: {message}'
if system:
    input = f'{input}\nSystem: {system}'

if input=='':
    prompt = prompt.format(instruction)
else:
    prompt = prompt.format("{}\n\n### input:\n{}""").format(instruction, input)

inputs = tokenizer(
    prompt,
    return_tensors="pt",
)
input_ids = inputs["input_ids"].cuda()

generation_config = GenerationConfig(
    do_sample=True #@param {type:"boolean"}
    ,temperature=0.4 #@param {type:"number"}
    ,top_p=0.65 #@param {type:"number"}
    ,top_k=45 #@param {type:"number"}
    # ,num_beams=1 #@param {type:"number"}
    ,repetition_penalty=1.05 #@param {type:"number"}
    # ,length_penalty=-0.25 #@param {type:"number"}
)

seed = 0 #@param {type:"integer"}
if seed == -1:
    seed = int.from_bytes(os.urandom(4), 'big')
manual_seed(seed)
print(f"Generating... Seed: {seed}")

print('='*40)
print('History: \n')
print('\n\n'.join(fixed_prompt+history[-(history_len*2):]))
print('='*40)
if input:
    print('Message:\n')
    print(input)
    print('='*40)

generation_output = model.generate(
    input_ids=input_ids,
    generation_config=generation_config,
    return_dict_in_generate=True,
    output_scores=True
    ,max_new_tokens=512 #@param {type:"integer"}
)

for s in generation_output.sequences:
    result = tokenizer.decode(s).strip()
    result = result.replace(prompt, '')
    if result=='':
        print('No output')
        print(prompt)
        print(result)
        continue
    print('\nResponse: ', end='')
    print(result)

    history += [f'User: {message}', f'Assistant: {result.strip()}']


<IPython.core.display.Javascript object>

Generating... Seed: 0
History: 


Message:

User: 你剛剛寫了什麼？

Response: 
我剛剛寫了一篇文章，講述了一個有趣的故事。


---
Normal instruction

In [ ]:
import os
from torch import manual_seed


prompt = """### Instruction:
{}

### Response:"""

instruction = "\u5199\u4E00\u5C01\u6C42\u804C\u4FE1" #@param {type:"string"}
input = "" #@param {type:"string"}

if input=='':
    prompt = prompt.format(instruction)
else:
    prompt = prompt.format("{}\n\n### input:\n{}""").format(instruction, input)

inputs = tokenizer(
    prompt,
    return_tensors="pt",
)
input_ids = inputs["input_ids"].cuda()

generation_config = GenerationConfig(
    do_sample=True #@param {type:"boolean"}
    ,temperature=0.5 #@param {type:"number"}
    ,top_p=0.75 #@param {type:"number"}
    ,top_k=45 #@param {type:"number"}
    # ,num_beams=1 #@param {type:"number"}
    ,repetition_penalty=1.05 #@param {type:"number"}
    # ,length_penalty=-0.25 #@param {type:"number"}
)

seed = -1 #@param {type:"integer"}
if seed == -1:
    seed = int.from_bytes(os.urandom(4), 'big')
manual_seed(seed)
print(f"Generating... Seed: {seed}")
print('='*40)
print('Instruction:')
print(instruction)

if input:
    print('\nInput:')
    print(input)

generation_output = model.generate(
    input_ids=input_ids,
    generation_config=generation_config,
    return_dict_in_generate=True,
    output_scores=True
    ,max_new_tokens=512 #@param {type:"integer"}
)

for s in generation_output.sequences:
    result = tokenizer.decode(s).strip()
    result = result.replace(prompt, '')
    if result=='':
        print('No output')
        print(prompt)
        print(result)
        continue
    print('\nResponse: ', end='')
    print(result)



<IPython.core.display.Javascript object>

Generating... Seed: 2818932291
Instruction:
写一封求职信

Response: 
尊敬的招聘经理，

我是一名有着丰富经验和成就的专业人士，对于您公司的职位招聘而言，我非常感兴趣。我在过去多年里在相关行业中取得了很多成功，并且在这个行业中具有深厚的知识和技能。

我认为我具有所需的技能和能力，并且擅长在各种不同的环境中实现出色。我的工作经验包括市场营销、品牌管理、客户服务等等。我曾在跨国公司担任过高级管理职位，并且在这些职位中表现出色。

我相信我是您公司需要的一员，并且我具有很好的沟通能力和团队合作能力。我也具有良好的沟通技巧和领导能力，能够协调和齐全团队。

我希望能够让您公司的业务进一步发展，并且为公司提供出色的服务。我相信我可以为您公司带来巨大的成功。

谢谢您对此的关注。如果您有任何问题或需要更多信息，请随时联系我。

谢谢！

祝您好运！

荣致！

张三
